# Requirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/NLP-project/

/content/drive/.shortcut-targets-by-id/1l0Urx0Wq9wfDemUffe3eYgmGJnGIHE7N/NLP-project


In [3]:
!pip install -q transformers datasets
!pip install -q selfcheckgpt
!pip install -U -q deepeval

!pip install faiss-gpu sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

# Dataset

In [4]:
# general packages
import torch
import random
import pandas as pd
from tqdm import tqdm

In [5]:
from datasets import load_dataset

dataset = load_dataset("SajjadAyoubi/persian_qa")
df = pd.DataFrame([example for example in dataset["train"]])
df = df.drop(columns=['id'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

# Wiki Data prepration (Do not run it)

In [ ]:
qa_titles = np.array(list(map(lambda x: x.replace('\u200c', ' '), qa_titles)))

In [ ]:
import os
import numpy as np
import pandas as pd

# Function to process and filter articles based on qa_titles
def filter_articles(file_path, qa_titles, mask):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    articles = content.split('عنوان مقاله:')
    filtered_articles = []
    for article in articles:
        if article.strip():  # Skip any empty strings
            lines = article.strip().split('\n', 1)
            title = lines[0].strip()
            if title in qa_titles:
                context = lines[1].strip() if len(lines) > 1 else ''
                filtered_articles.append({'title': title, 'context': context})
                mask[np.where(qa_titles == title)[0][0]] = False

    return filtered_articles, len(filtered_articles), mask

# Function to merge filtered content from all files into a DataFrame
def merge_files(directory_path, qa_titles):
    c = 0
    mask = np.ones(len(qa_titles), dtype=bool)
    filtered_df = pd.DataFrame(columns=['title', 'context'])

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            filtered_articles, n, mask = filter_articles(file_path, qa_titles, mask)
            c += n
            if filtered_articles:
                df = pd.DataFrame(filtered_articles)
                filtered_df = pd.concat([filtered_df, df], ignore_index=True)

    print(c)
    return filtered_df, mask

# Specify the directory containing the text files and the output file path
directory_path = '/content/drive/MyDrive/NLP-project/persian-wikipedia-dataset/'  # Replace with your actual directory path
output_csv_path = '/content/drive/MyDrive/NLP-project/selected_articles.csv'  # Replace with the desired output CSV file path

# Merge all files and get the DataFrame
filtered_df, mask = merge_files(directory_path, qa_titles)

# Save the DataFrame to a CSV file
filtered_df.to_csv(output_csv_path, index=False, encoding='utf-8')

# Optionally print the titles that were not found
unfound_titles = qa_titles[mask]
print(f"Titles not found: {unfound_titles}")


865
Titles not found: ['وزنه برداری قدرتی' 'دانشگاه های قرون وسطی' 'هم حسی' 'ایران ناسیونال'
 'توئیتر' 'فریزر' 'C++\u200e' 'کهکشان راه شیری' 'نیویورک (ایالت)' 'طیاره'
 'گردو' 'خمیر دندان' 'تاریخچهٔ بوکس' 'رانی کولمن' 'هلیم' 'بی تی اس'
 'ریحانه پارسا' 'فخریه اوجن' 'کیم یو-جونگ' 'لئوناردو داوینچی' 'قورمه سبزی'
 'هواگرد جنگنده' 'بادمجان' 'چالش مومو' 'افسانه جومونگ' 'سیب زمینی'
 'دریای خزر 2' 'دگمه' 'قوس' 'بازیگر' 'گوجه' 'ژانر موسیقی' 'آموزش'
 'الکتریسیته']


In [ ]:
wiki_titles = np.array(list(map(lambda x: x.replace(' ', '_'), qa_titles)))

In [ ]:
wiki_titles

array(['شرکت_فولاد_مبارکه_اصفهان', 'علوم_رایانه', 'ربات', 'سیب', 'کتاب',
       'گل', 'باشگاه_فوتبال_منچستر_یونایتد', 'اسکات', 'پهپاد',
       'وزنه_برداری_قدرتی', 'نوکیا', 'ایران_خودرو', 'دانشگاه',
       'گرمایش_جهانی', 'حساب_سپرده', 'تهران', 'دکترا', 'تخت_جمشید',
       'هری_فرانکفورت', 'کاغذ', 'لینوکس', 'مریم_میرزاخانی',
       'شاهنشاهی_ساسانی', 'راه_شیری', 'شبه_علم', 'فناوری_هسته_ای',
       'پاستای_گولاش_آمریکایی', 'پارچه', 'بستنی', 'اینترنت', 'آزمون_آلن',
       'آزمون_تورینگ', 'دانشگاه_های_قرون_وسطی', 'کارل_پوپر', 'گوگل',
       'رباتیک', 'گرانش', 'تسلا_موتورز', 'هواپیما', 'کتاب_الکترونیک',
       'دروغ_بزرگ', 'باشگاه_مبارزه_(رمان)', 'کودک_همسری', 'وینستون_چرچیل',
       'معماری_آرم', 'تمدن_مایا', 'اسپیس_اکس', 'تلویزیون', 'برنامه_ریزی',
       'بهره_هوشی', 'انسان', 'حداقل_دستمزد', 'توزیع_درآمد', 'بیت_کوین',
       'هم_حسی', 'اخترشناسی', 'چرم', 'طلا', 'پول', 'یخچال', 'شکر', 'قهوه',
       'آیرودینامیک', 'امنیت_شبکه', 'مریخ', 'اسکان_بشر_در_مریخ',
       'ایران_ناسیونال', 'سرمایه

In [ ]:
import requests
crawl = {}
for t in wiki_titles:
  url = f"https://fa.wikipedia.org/wiki/{t}"
  response = requests.get(
    url=url,
  )
  if response.status_code == 200:
    crawl[t] = response.text
  else:
    print(t)

دانشگاه_های_قرون_وسطی
تاریخچهٔ_بوکس
آتش_سوزی_و_ریزش_ساختمان_پلاسکو
مهدی_شیری
دریای_خزر_2


In [ ]:
crawl['سردار_آزمون']

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="fa" dir="rtl">\n<head>\n<meta charset="UTF-8">\n<title>سردار آزمون - ویکی\u200cپدیا، دانشنامهٔ آزاد</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-di

In [ ]:
import csv
# Specify the filename
filename = '/content/drive/MyDrive/NLP-project/selected_wiki.csv'

# Write the dictionary to the CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Title', 'Context'])
    # Write the data
    for title, context in crawl.items():
        writer.writerow([title, context])

## Embedding

In [ ]:
https://www.kaggle.com/code/ttminh27/rag-using-bert-base-and-mistral-base

# Wiki dataset (do not run this)

In [ ]:
wiki_df = pd.read_csv("/content/drive/MyDrive/NLP-project/selected_articles.csv")

In [ ]:
print(df.shape)
print(wiki_df.shape)

(9008, 4)
(865, 2)


In [ ]:
import pandas as pd
import random
random.seed(42)

# Ensure the selected titles are present in both wiki_df and df
available_titles_in_df = pd.unique(df["title"])  # Get unique titles in df
available_titles_in_wiki = pd.unique(wiki_df["title"])  # Get unique titles in wiki_df

# Get intersection of available titles
common_titles = list(set(available_titles_in_df) & set(available_titles_in_wiki))

# Adjust k to select the minimum between 100 and the number of common titles
k = min(100, len(common_titles))

# Initialize an empty list to store unique titles
selected_titles = []

# Continue selecting titles until we have 100 unique ones in the dataset
while len(selected_titles) < 100:
    # Select k titles from the common titles
    random.seed(42)
    random_selection = random.sample(common_titles, k=k)

    # Filter df based on the current random selection
    temp_dataset = df.loc[df["title"].isin(random_selection)]

    # Extract the unique titles from the filtered dataset
    unique_titles_in_temp = pd.unique(temp_dataset['title'])

    # Add these unique titles to the selected titles list
    selected_titles.extend(unique_titles_in_temp)

    # Deduplicate the selected titles list
    selected_titles = list(set(selected_titles))

    # Adjust k for the next iteration
    k = 100 - len(selected_titles)

    # Break if the total number of available titles is exhausted
    if k <= 0 or len(selected_titles) >= len(common_titles):
        break

# Filter the df dataset based on the selected titles
dataset = df.loc[df["title"].isin(selected_titles[:100])]

dataset.reset_index(drop=True, inplace=True)

dataset.shape

(1010, 4)

In [ ]:
dataset

,title,context,question,answers
0,کاغذ,کاغذ (به عربی: ورق)، به ماده‌ای گفته می‌شود که...,به چه ماده ای کاغذ گفته می شود؟,{'text': ['از خمیر کتان یا پنبه یا کنف یا چوب ...
1,کاغذ,کاغذ (به عربی: ورق)، به ماده‌ای گفته می‌شود که...,کاغذ برای بشر چه اهمیتی دارد؟,{'text': ['جمع‌آوری و ذخیره اطلاعات و دانش بشر...
2,کاغذ,کاغذ (به عربی: ورق)، به ماده‌ای گفته می‌شود که...,استفاده از کاغذ از چه زمانی شروع شد؟,"{'text': ['با اختراع آن'], 'answer_start': [345]}"
3,کاغذ,کاغذ (به عربی: ورق)، به ماده‌ای گفته می‌شود که...,اولین بار چه کسانی از کاغذ استفاده کردند؟,"{'text': ['مصریان'], 'answer_start': [388]}"
4,کاغذ,کاغذ (به عربی: ورق)، به ماده‌ای گفته می‌شود که...,مصری ها از چه درختی کاغذ درست می کردند؟,"{'text': ['درختی به نام (پاپیروس)'], 'answer_s..."
...,...,...,...,...
1005,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علوم اجتماعی چیه؟,{'text': ['دسته‌ای از تخصص‌های آکادمیک است، که...
1006,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علم عمومی جامعه چیه؟,"{'text': ['جامعه‌شناسی'], 'answer_start': [483]}"
1007,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,دانشمندای تفسیر گرا چه روشی ندارن؟,"{'text': [], 'answer_start': []}"
1008,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علوم اجتماعی چه زیر شاخه هایی نداره؟,"{'text': [], 'answer_start': []}"


In [ ]:
len(pd.unique(dataset['title']))

100

In [ ]:
mini_wiki_df = wiki_df[wiki_df['title'].isin(pd.unique(dataset['title']))]

# mini wiki final

In [42]:
mini_wiki_df = pd.read_csv("./mini_wiki_df.csv")
selected_titles = mini_wiki_df["title"]
dataset = df.loc[df["title"].isin(selected_titles)].reset_index(drop=True)
wiki_texts = mini_wiki_df['context']

# Base Model

In [57]:
# from base_models import get_base_model
# base_model = get_base_model(model_name = "bert-base-multilingual-xquad")


# from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
# model_name_path = "bert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_name_path)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name_path)
# base_model = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

# from transformers import AutoConfig, AutoTokenizer, AutoModel
# config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# base_model = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

# from transformers import pipeline
# base_model = pipeline("question-answering", model="marzinouri/parsbert-finetuned-persianQA",  device=0)


from transformers import AutoTokenizer, AutoModelForQuestionAnswering
path = 'pedramyazdipoor/parsbert_question_answering_PQuAD'
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForQuestionAnswering.from_pretrained(path)
base_model = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)


In [ ]:
num_sample = 1

for sample_idx in range(num_sample):
    model_output = []
    for index, element in tqdm(dataset.iterrows()):
        QA_input = {
            'question': element['question'],
            'context': element['context']
        }
        result = base_model(QA_input)

        if result['score'] < 0.1:
            # Append the generated answer
            generated_answer = "پاسخ ندارد"
        else:
            # Append the generated answer
            generated_answer = result['answer']


        model_output.append(generated_answer)

    dataset['model_output' + str(sample_idx)] = model_output

8it [00:01,  8.15it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
1010it [00:20, 49.08it/s]
<ipython-input-15-63b786adab50>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['model_output' + str(sample_idx)] = model_output


In [ ]:
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available
dataset["selfcheckscore0"] = None
for index, example in tqdm(dataset.iterrows()):
    model_output0 = example["model_output0"]

    if example["answers"]['text'] == []:
        main_answer = "پاسخ ندارد"
    else:
        main_answer = example["answers"]['text'][0]

    sent_scores_nli = selfcheck_nli.predict(
        sentences = [model_output0],   # model output
        sampled_passages = [main_answer], # main answer
    )
    dataset.loc[index, "selfcheckscore0"] = list(sent_scores_nli)[0] # Store the scores in the sent_scores_nli column


<ipython-input-16-ecf66bcbf998>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["selfcheckscore0"] = None


SelfCheck-NLI initialized to device cuda


1010it [00:47, 21.14it/s]


In [ ]:
dataset['selfcheckscore0'].mean()

0.6948886979998339

In [ ]:
dataset

,title,context,question,answers,model_output0,selfcheckscore0,model_output1,selfcheckscore1,retrieved_text
0,گل,یک گل، از چهار قسمت که به بالای یک ساقه کوتاه ...,گل چیه؟,{'text': ['یک گل، از چهار قسمت که به بالای یک ...,پاسخ ندارد,0.998178,پاسخ ندارد,0.998178,ی (یک خوشه گرانش، شامل کهکشان های مقید، در میا...
1,گل,یک گل، از چهار قسمت که به بالای یک ساقه کوتاه ...,کاسبرگ‌ها چکار می‌کنند؟,{'text': ['به‌عنوان پوششی از تبخیر سریع جلوگیر...,پاسخ ندارد,0.997796,پاسخ ندارد,0.997796,خطر انتقال از طریق تزریق در آفریقا را ۱٫۲٪ تخ...
2,گل,یک گل، از چهار قسمت که به بالای یک ساقه کوتاه ...,اجزای گیاهی گل کدامند؟,"{'text': ['کاسه گل و جام گل'], 'answer_start':...",کاسه گل و جام گل,0.043425,پاسخ ندارد,0.998662,کیوی گیاهی است از گروه گیاهان گلدار دولپه ای ه...
3,گل,یک گل، از چهار قسمت که به بالای یک ساقه کوتاه ...,تعریف پریانت چیست؟,{'text': ['به مجموعهٔ کاسبرگ‌ها و گلبرگ‌ها پری...,پاسخ ندارد,0.999349,پاسخ ندارد,0.999349,خطر انتقال از طریق تزریق در آفریقا را ۱٫۲٪ تخ...
4,گل,یک گل، از چهار قسمت که به بالای یک ساقه کوتاه ...,گلبرگ‌ها چه وظیفه‌ای دارند؟,{'text': ['با رنگ‌های جذاب خود حشرات را به غدد...,مجموعاً جام‌گل گفته می‌شوند.,0.924808,پاسخ ندارد,0.997744,ید به زندگی در این کشور ۷۸/۷ سال است.غالب قطری...
...,...,...,...,...,...,...,...,...,...
995,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علوم اجتماعی چیه؟,{'text': ['دسته‌ای از تخصص‌های آکادمیک است، که...,پاسخ ندارد,0.999421,پاسخ ندارد,0.999421,خطر انتقال از طریق تزریق در آفریقا را ۱٫۲٪ تخ...
996,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علم عمومی جامعه چیه؟,"{'text': ['جامعه‌شناسی'], 'answer_start': [483]}",پاسخ ندارد,0.942116,پاسخ ندارد,0.942116,غلب به طور سازمان یافته به سه حوزه وسیع تقسیم ...
997,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,دانشمندای تفسیر گرا چه روشی ندارن؟,"{'text': [], 'answer_start': []}",پاسخ ندارد,0.089584,پاسخ ندارد,0.089584,"ژوهش های ادبی فمینیستی غربی، مطالعاتی چون ""ماد..."
998,علوم اجتماعی,علوم اجتماعی، دسته‌ای از تخصص‌های آکادمیک است،...,علوم اجتماعی چه زیر شاخه هایی نداره؟,"{'text': [], 'answer_start': []}",پاسخ ندارد,0.089584,پاسخ ندارد,0.089584,خطر انتقال از طریق تزریق در آفریقا را ۱٫۲٪ تخ...


# RAG

In [30]:
import faiss
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

In [ ]:
import pandas as pd
mini_wiki_df = wiki_df[wiki_df['title'].isin(pd.unique(dataset['title']))]
# Convert 'full_context' to string and handle NaN values by filling them with an empty string
mini_wiki_df['context'] = mini_wiki_df['context'].fillna("").astype(str)

# Now create the list of contexts
wiki_texts = mini_wiki_df['context']

<ipython-input-149-20f789a7abde>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_wiki_df['context'] = mini_wiki_df['context'].fillna("").astype(str)


In [43]:
all_chunks = []
chunk_size = 500
overlap = 100
for text in tqdm(wiki_texts.tolist()):
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        start += (chunk_size - overlap)

        if len(chunk) > 300:
            all_chunks.append(chunk.replace("\n" , ""))
all_chunks = pd.DataFrame(all_chunks, columns = ['text'])

100%|██████████| 100/100 [00:00<00:00, 20011.95it/s]


In [44]:
# Check if GPU is available and use it for embedding model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [45]:
encoder = SentenceTransformer('paraphrase-MiniLM-L6-v2', device=device)
# encoder = SentenceTransformer('distiluse-base-multilingual-cased', device=device)
# encoder = SentenceTransformer('paraphrase-xlm-r-multilingual-v1', device=device)
wiki_embeddings = encoder.encode(all_chunks['text'], show_progress_bar=True , device=device)

Batches:   0%|          | 0/72 [00:00<?, ?it/s]

In [46]:
import time
from sentence_transformers import SentenceTransformer, util

# Function that searches the corpus and prints the results
def search(inp_question, num_res):
    res = {}
    start_time = time.time()
    question_embedding = encoder.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, wiki_embeddings)
    end_time = time.time()
    hits = hits[0]  #Get the hits for the first query

    # print("Input question:", inp_question)
    # print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:num_res]:

        res[hit['corpus_id']] = all_chunks['text'][hit['corpus_id']]
    df = pd.DataFrame(list(res.items()),columns = ['id','res'])
    return df

In [38]:
import faiss

vector_dimension = wiki_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(wiki_embeddings)
faiss_index.add(wiki_embeddings)

# nlist = 100  # Number of clusters
# quantizer = faiss.IndexFlatL2(vector_dimension)
# faiss_index = faiss.IndexIVFFlat(quantizer, vector_dimension, nlist, faiss.METRIC_L2)
# faiss_index.train(wiki_embeddings)  # Train on your embeddings
# faiss_index.add(wiki_embeddings)

# faiss_index = faiss.IndexHNSWFlat(vector_dimension, 32)  # 32 is the number of neighbors
# faiss_index.add(wiki_embeddings)

# m = 16  # Number of subquantizers
# nbits = 8  # Number of bits per subquantizer
# faiss_index = faiss.IndexPQ(vector_dimension, m, nbits)
# faiss_index.train(wiki_embeddings)
# faiss_index.add(wiki_embeddings)

In [58]:
num_sample = 1

for sample_idx in range(num_sample):
    model_output = []
    k = 0
    for index, element in tqdm(dataset.iterrows()):
        question = element['question']
        # print()
        # print(element['title'])
        # print(element['context'])
        # print( element['question'])
        # print(element['answers'])
        # print("*************************")

        '''
        # if metric is FAISS uncomment this
        question_embedding = encoder.encode(question)


        _vector = np.array([question_embedding])
        faiss.normalize_L2(_vector)

        # Search the FAISS index for the most similar contexts
        distances, indices = faiss_index.search(_vector, k = 3)


        # Retrieve the corresponding contexts
        relevant_contexts = [all_chunks['text'].iloc[idx] for idx in indices[0]]
        # Combine the retrieved contexts into a single string
        retrieved_context = " ".join(relevant_contexts)
        # print(retrieved_context)

        '''


        retrieved_context = search(question, 1)

        # Combine original context with the retrieved context
        combined_context = element['context'] + "\n" + retrieved_context.iloc[0]['res']
        # Create input for the model
        QA_input = {
            'question': question,
            'context': combined_context
        }
        dataset.loc[index, 'retrieved_text'] = retrieved_context

        # Get the result from the model
        result = base_model(QA_input)
        # print(result['score'] , result['answer'] )
        if result['score'] < 0.1:
            # Append the generated answer
            generated_answer = "پاسخ ندارد"
        else:
            # Append the generated answer
            generated_answer = result['answer']

        # print(generated_answer)

        # print("**********************************************************")
        # print()
        model_output.append(generated_answer)

        # k+=1
        # if k > 10:
        #     break

    # Store the model output in the dataset
    dataset['model_output1'] = model_output

# At this point, dataset includes the model outputs after RAG using FAISS retrieval


1000it [00:34, 29.13it/s]


In [59]:
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available
dataset["selfcheckscore1"] = None
for index, example in tqdm(dataset.iterrows()):
    model_output1 = example["model_output1"]

    if example["answers"]['text'] == []:
        main_answer = "پاسخ ندارد"
    else:
        main_answer = example["answers"]['text'][0]

    sent_scores_nli = selfcheck_nli.predict(
        sentences = [model_output1],   # model output
        sampled_passages = [main_answer], # main answer
    )
    dataset.loc[index, "selfcheckscore1"] = list(sent_scores_nli)[0] # Store the scores in the sent_scores_nli column


SelfCheck-NLI initialized to device cuda


1000it [00:48, 20.73it/s]


In [60]:
print(dataset['selfcheckscore0'].mean())
print(dataset['selfcheckscore1'].mean())

0.700247547596693


In [ ]:
dataset.head(30)

In [ ]:
dataset.iloc[-1]['retrieved_text']